In [5]:
import os
import astropy.io.fits as fits
import datetime as dt
import gzip
import re
import glob
from pprint import pprint as pp

In [6]:
root_dir = "/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test"
ALLOWED_EXT = ['.gz', '.bz2']
forbidden_keys = ['HISTORY', 'COMMENT']

In [7]:
def get_dirs_to_walk(root_dir):
    folders_in_root = os.listdir(root_dir)
    p = re.compile('^\d{4}-\d{2}-\d{2}$')
    
    dirs_to_walk = []
    for _d in os.listdir(root_dir):
        if p.match(_d):
            dirs_to_walk.append(os.path.join(root_dir, _d))
    return dirs_to_walk


def get_files(_dir):
    files_in_folder = sorted(glob.glob(_dir + '/*'))
    files_to_open = []
    
    for _file in files_in_folder:
        if os.path.isfile(_file):
            if os.path.splitext(_file)[-1] in ALLOWED_EXT:
                files_to_open.append(_file)
    
    return files_to_open


def get_folder_data(files_to_open):
    folder_data = []
    for f in files_to_open:
        with gzip.open(f, 'rb') as f_in:
            hdr = dict(fits.getheader(f))
            
            obs_datetime = dt.datetime.strptime(
                hdr['DATE-OBS'] + 'T' + hdr['TIME-OBS'],
                '%Y-%m-%dT%H:%M:%S.%f')
            
            object_name = hdr['OBJECT']
            observer = hdr['OBSERVER']
            color_filter = hdr['FILTER']
            exptime = hdr['EXPTIME']
            
        row = {
            'obs_datetime': obs_datetime,
            'object_name': object_name,
            'observer': observer,
            'color_filter': color_filter,
            'exptime': exptime,
        }
        folder_data.append(row)
        
    folder_data = sorted(folder_data, key=lambda x: x['obs_datetime'])
    
    return folder_data


def get_grouped_folder_data(folder_data):
    results = []
    target = {}
    n_frames = 0
    prev_name = None

    for frame in folder_data:
        if frame['object_name'] != prev_name:
            n_frames = 0
            if target:
                target['datetime_end'] = frame['obs_datetime']
                results.append(target)
            target = {}
            prev_name = frame['object_name']
            target['name'] = frame['object_name']
            target['datetime_start'] = frame['obs_datetime']
            target['observer'] = frame['observer']
            target['colorfilters'] = []
            target['total_exposure_time'] = 0
            target['number_of_frames'] = 0

        target['number_of_frames'] += 1    
        target['total_exposure_time'] += float(frame.get('exptime', 0))
        if frame['color_filter'] not in target['colorfilters']:
            target['colorfilters'].append(frame['color_filter'])

    return results
    


In [8]:
dirs_to_walk = get_dirs_to_walk(root_dir)
for _dir in dirs_to_walk:
    print('#'*9)
    pp(_dir)
    folder_files = get_files(_dir)
    folder_data = get_folder_data(folder_files)
    grouped_folder_data = get_grouped_folder_data(folder_data)
    pp(grouped_folder_data)
    print('#'*9)

#########
'/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/2015-10-10'
[{'colorfilters': ['GS', 'IS'],
  'datetime_end': datetime.datetime(2015, 10, 10, 21, 23, 51, 130000),
  'datetime_start': datetime.datetime(2015, 10, 10, 20, 47, 21, 400000),
  'name': 'lkha',
  'number_of_frames': 40,
  'observer': 'siwak',
  'total_exposure_time': 1000.0},
 {'colorfilters': ['GS', 'IS'],
  'datetime_end': datetime.datetime(2015, 10, 10, 22, 3, 39, 840000),
  'datetime_start': datetime.datetime(2015, 10, 10, 21, 23, 51, 130000),
  'name': 'ngc7129',
  'number_of_frames': 61,
  'observer': 'siwak',
  'total_exposure_time': 1830.0}]
#########
#########
'/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/2017-12-16'
[{'colorfilters': ['R'],
  'datetime_end': datetime.datetime(2017, 12, 17, 0, 52, 21, 360000),
  'datetime_start': datetime.datetime(2017, 12, 17, 0, 16, 40, 720000),
  'name': 'j0952',
  'number_of_frames': 18,
  'observer': 'MS',
  'total_exposure_time': 1080.0}